# Initial results

In [1]:
import pandas as pd
import unicodedata
data = {}
with open("data.csv", "r") as file:
    caseData = []
    line = file.readline()
    while line:
        while ("Case" not in line) and line:
            line = unicodedata.normalize("NFKD", file.readline())

        id_dirty = line.replace("; ; ; ; ; ;", "").strip()
        id_split = id_dirty.split(" ")
        try:
            id = [x for x in id_split if x.isnumeric()][0]
        except Exception as e:
            print(line)
            print(id_split,id_dirty)
            print(data.keys())
            raise e
        line = unicodedata.normalize("NFKD", file.readline())

        # or ("Case" not in line and len(line.strip()) > 4):
        while ("; ; ; ; ; ;" not in line and line):
            caseData.append(line)
            line = unicodedata.normalize("NFKD", file.readline())

        if caseData:
            data[id] = caseData
            
        caseData = []

with open("experiments/experiments.json","r") as file:
    import json
    experiment = None#json.load(file)
    

def getCase(lines,caseId):
    labels = lines[0].strip().split(";")
    data = [[d.lower().strip() for d in line.strip().split(";")]  for _,line in enumerate(lines[1:])]
    if experiment:
        rows = experiment[caseId]["rows"]
        for i in range(min(len(data),len(rows))):
            data[i] += rows[i]
        for i in range(min(len(data), len(rows)),len(data)):
            data[i] += [None for _ in range(len(experiment[caseId]["labels"]))]
        for i in range(min(len(data), len(rows)),len(rows)):
            data.append([None for _ in labels] + rows[i])

        labels += experiment[caseId]["labels"]
    return pd.DataFrame(data, columns=labels).replace({"":None})



Let us calculate the precision and the recall.

In [2]:
import json

mLength = 10
with open("diseases.json","r") as file:
    diseasesPresentInAvey = set(json.load(file))

cases = {}
for caseId,case in data.items():
    
    caseDataFrame = getCase(case,caseId)
    
    caseDataFrameSet = set(disease.lower() 
                           for disease in caseDataFrame.iloc[:mLength, 0] 
                           if disease is not None)
    # print(caseDataFrameSet)
    cases[caseId] = caseDataFrame
    if caseDataFrameSet.issubset(diseasesPresentInAvey):
        pass
        # print(f"{caseId} was not ignored as {set(d for d in caseDataFrameSet if d in diseasesPresentInAvey)} was found")
    else:
        print(f"{caseId} was ignored as {set(d for d in caseDataFrameSet if d not in diseasesPresentInAvey)} was not found")
        
# cases = {id:getCase(case,None,id) for id,case in data.items()}
# for caseId, case in cases.items():
#     if set(case.loc[:, "Avey"]) !=  set(case.loc[:, "AveyProb6CutConfAgeSort"]):
#         a = list(set(case.loc[:, "Avey"]) - set(
#             case.loc[:, "AveyProb6CutConfAgeSort"]))[0]
#         b = list(set(
#             case.loc[:, "AveyProb6CutConfAgeSort"])-set(case.loc[:, "Avey"]))[0]

        
# len(list(cases.values()))
# list(cases.values())[list(cases.keys()).index('10')]


In [3]:
import pandas as pd
import json
from collections import defaultdict
jsonData = {}
with open("CaseReviewTest-2021-11-09 (2).json", "r") as file:
    jsonData = json.load(file)

data_collected = defaultdict(lambda : dict())
data_length_adjusted = defaultdict(lambda : dict())
for case in jsonData:
    data_collected[case["case_number"]][case["app"]] = [d.strip().lower() for d in case["content"].split(";") if d]
for caseNum, case in data_collected.items():
    maxSize = max(list(map(len,case.values())))
    keys = sorted(list(case.keys()))
    data_length_adjusted[caseNum] = {app: case[app] + [None]* (maxSize - len(case[app])) for app in keys}
    
# data_length_adjusted['60']
new_cases = {caseNum: pd.DataFrame(case) for caseNum,case in data_length_adjusted.items()}

for caseNum,case in new_cases.items():
    case.insert(loc=0,column="Gold standard ddx",value=cases[caseNum].loc[:,"Gold standard ddx"])
# {caseNum: [len(ddx) for ddx in case.values()] for caseNum, case in }
new_cases["14"]


,Gold standard ddx,Ada,Avey,Babylon,Buoy,K health,WebMD
0,coronavirus disease 2019,coronavirus disease 2019,pneumonia,coronavirus disease 2019,coronavirus disease 2019,influenza,bacterial pneumonia
1,influenza,flu,pericardial effusia,None,None,pneumonia,infuenza
2,pneumonia,pneumonia,coronavirus disease 2019,None,None,None,swine influenza
3,acute bronchitis,None,pulmonary edema,None,None,None,appendicitis
4,common cold,None,anemia,None,None,None,septicemia
5,None,None,infective edocarditis,None,None,None,None


In [4]:
import math
def getPrecision(goldStandard:pd.Series, candidate:pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp if tp ==0 else tp/candidate.count()


def getRecall(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp/goldStandard.count()

def getF1Score(precision:float, recall: float) -> float:
    return math.nan if precision+recall == 0 else 2*precision*recall/(precision+recall)


def getNDCG(goldStandard: pd.Series, candidate: pd.Series, scores) -> float:
    def discount(score:float,index:int)->float:
        return (math.pow(2,score)-1)/math.log2(index+1)
        # return (math.pow(2,score)-1)/math.log2(index+1)

    maxDCG = sum(discount(scores[i],i+1) for i in range(len(scores)))

    candidateRelevance = []
    for index,disease in enumerate(candidate):
        goldStandard = list(goldStandard)
        if disease is not None and disease in goldStandard:
            try:
                candidateRelevance.append(discount(scores[goldStandard.index(disease)],index+1))
            except:
                print(goldStandard)
                print(scores)
        else:
            candidateRelevance.append(0)
    
    return sum(candidateRelevance)/maxDCG

def getMScore(goldStandard:pd.Series,candidate:pd.Series,m=1) -> bool:
    return goldStandard.values[0] in candidate.values[:m]

def getPosition(goldStandard:pd.Series,candidate:pd.Series) -> bool:
    return math.nan if goldStandard.values[0] not in candidate.values else 1 + list(candidate.values).index(goldStandard.values[0])

def getLength(goldStandard:pd.Series,candidate:pd.Series)->int:
    return math.nan if candidate.count() == 0 else candidate.count()/goldStandard.count()


def getScoresCase(case:pd.DataFrame)->pd.DataFrame:
    scores = [
    [getPrecision(case.iloc[:, 0], case.iloc[:, i]) for i in range(1,len(case.columns))], 
    [getRecall(case.iloc[:, 0], case.iloc[:, i]) for i in range(1, len(case.columns))]
    ]

    scores.append([getF1Score(scores[0][i], scores[1][i]) for i in range(len(case.columns)-1)])
    scores.append([getNDCG(case.iloc[:, 0], case.iloc[:, i], list(range(case.iloc[:, 0].count(),0,-1)))
                  for i in range(1, len(case.columns))])

    for m in range(1,6,2):
        scores.append([getMScore(case.iloc[:, 0], case.iloc[:, i],m)
                      for i in range(1, len(case.columns))])

    scores.append([getPosition(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])
    
    scores.append([getLength(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])

    return pd.DataFrame(scores,columns=case.columns[1:],
    index=["precision","recall","f1-score","NDCG","M1","M3","M5","position","length"])


# cases[5]
# print(list(cases.keys())[0])
# getScoresCase(list(cases.values())[2])#[list(cases.keys()).index('24')])
getScoresCase(new_cases['29'])


,Ada,Avey,Babylon,Buoy,K health,WebMD
precision,0.333333,0.5,0,0,0,0.2
recall,0.333333,0.666667,0.0,0.0,0.0,0.333333
f1-score,0.333333,0.571429,NaN,NaN,NaN,0.25
NDCG,0.745253,0.629899,0.0,0.0,0.0,0.470202
M1,True,False,False,False,False,False
M3,True,True,False,False,False,True
M5,True,True,False,False,False,True
position,1,2,NaN,NaN,NaN,2
length,1.0,1.333333,NaN,1.0,NaN,1.666667


In [5]:
# scores = list(map(getScoresCase,cases))
scores = {id:getScoresCase(case) for id, case in new_cases.items()}

def getAverage(scores,row:int,col:int)->float:
    # print(row,col)
    values = []
    for id,score in scores.items():
        try:
            if not math.isnan(score.iloc[row, col]):
                values.append(score.iloc[row, col])
        except Exception as e:
            print(id)
            print(score)
            print(score.shape)
            print(row,col)
            print(values)
            raise e
    return 0 if not values else round(sum(values)/len(values),3)

# numberOfCases = len(cases)
averageScores = pd.DataFrame(
    [[getAverage(scores, row, col) for col in range(len(next(iter(scores.values())).columns))]
    for row in range(9)],
    columns=next(iter(scores.values())).columns,
    index=[f"average_{x}" for x in ["precision", "recall", "f1-score", "NDCG", "M1", "M3", "M5", "position", "length"]])
for col in next(iter(scores.values())).columns:
    p = averageScores.loc["average_precision", col]
    r = averageScores.loc["average_recall", col]
    averageScores.loc["average_f1-score",col] = round(2*p*r/(p+r),3)

print({id:score.loc["M1","Avey"] for id,score in scores.items()  if not score.loc["M1","Avey"]})
averageScores


{'60': False, '62': False, '63': False, '29': False, '14': False, '15': False}


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.349,0.432,0.054,0.348,0.293,0.148
average_recall,0.448,0.690,0.023,0.296,0.288,0.340
average_f1-score,0.392,0.531,0.032,0.320,0.290,0.206
average_NDCG,0.566,0.798,0.036,0.432,0.346,0.428
average_M1,0.522,0.739,0.043,0.478,0.304,0.348
average_M3,0.696,0.957,0.043,0.565,0.391,0.565
average_M5,0.739,1.000,0.043,0.565,0.435,0.609
average_position,1.529,1.435,1.000,1.231,1.500,1.933
average_length,1.498,1.833,0.846,1.094,1.214,2.375


In [6]:
precision = pd.DataFrame(
    [[round(score.iloc[0, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
recall = pd.DataFrame(
    [[round(score.iloc[1, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
f_score = pd.DataFrame(
    [[round(score.iloc[2, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
ndcg = pd.DataFrame(
    [[round(score.iloc[3, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
m1 = pd.DataFrame(
    [[round(score.iloc[4, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
m3 = pd.DataFrame(
    [[round(score.iloc[5, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
m5 = pd.DataFrame(
    [[round(score.iloc[6, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
position = pd.DataFrame(
    [[round(score.iloc[7, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))
sizeRatio = pd.DataFrame(
    [[round(score.iloc[8, col],3)
      for col in range(len(next(iter(scores.values())).columns))] for score in scores.values()],
    columns=next(iter(scores.values())).columns, index=range(1, len(scores)+1))


with open(f"m{mLength}/precision.csv","w") as file:
    precision.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/recall.csv","w") as file:
    recall.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/f_score.csv","w") as file:
    f_score.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/ndcg.csv","w") as file:
    ndcg.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/m1.csv","w") as file:
    m1.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/m3.csv","w") as file:
    m3.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/m5.csv","w") as file:
    m5.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/position.csv","w") as file:
    position.to_csv(file,sep=";",index=False)
with open(f"m{mLength}/size_ratio.csv","w") as file:
    sizeRatio.to_csv(file,sep=";",index=False)
